[View in Colaboratory](https://colab.research.google.com/github/djinnome/jmvae/blob/master/MOMA.ipynb)

In [2]:
!git clone https://github.com/djinnome/MOMA.git

Cloning into 'MOMA'...
remote: Counting objects: 87, done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 87 (delta 43), reused 63 (delta 23), pack-reused 0
Unpacking objects: 100% (87/87), done.


# MOMA

## Steps to run MOMA
### Step 1: Install software requirements

* R 3.4.0 or above
* Python 3.6.3 or above
* Python TensorFlow package
* Python numpy package
* Python pandas package

Tip: once you install python TensorFlow, you can simply install all other required python packages by
<code>pip install -r requirements.txt</code>.


In [3]:
!cd MOMA && pip install -r requirements.txt

    100% |████████████████████████████████| 12.1MB 1.8MB/s 
    100% |████████████████████████████████| 22.4MB 1.2MB/s 
  Found existing installation: numpy 1.14.5
    Uninstalling numpy-1.14.5:
      Successfully uninstalled numpy-1.14.5
  Found existing installation: pandas 0.22.0
    Uninstalling pandas-0.22.0:
      Successfully uninstalled pandas-0.22.0



### Step 2: Download the dataset
Then you download the Ecomcis transcriptome data from [here](https://www.dropbox.com/sh/t3zs3jbmq1efj3q/AAATQNlJimWT1bnTI9uK81S9a?dl=0) and place it in Dataset folder.


In [4]:
!cd MOMA/Dataset && wget -c https://www.dropbox.com/sh/t3zs3jbmq1efj3q/AAD4KYrRuLfPFJBQXvbA1yuDa/Ecomics.transcriptome.no_avg.v8.txt

--2018-07-13 18:58:17--  https://www.dropbox.com/sh/t3zs3jbmq1efj3q/AAD4KYrRuLfPFJBQXvbA1yuDa/Ecomics.transcriptome.no_avg.v8.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.1, 2620:100:601f:1::a27d:901
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/raw/t3zs3jbmq1efj3q/AAD4KYrRuLfPFJBQXvbA1yuDa/Ecomics.transcriptome.no_avg.v8.txt [following]
--2018-07-13 18:58:17--  https://www.dropbox.com/sh/raw/t3zs3jbmq1efj3q/AAD4KYrRuLfPFJBQXvbA1yuDa/Ecomics.transcriptome.no_avg.v8.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc601df3b677309595e0dfbb3289.dl.dropboxusercontent.com/cd/0/inline/ALRXygVLFesfgb-igq0EOAm4fCSVHnuoML62Dlhsp2OQivJxRSIRIDjLrvw40-TyiFMdyGoTxGJYQ9o3_lYRAryLhcWDssEKOH5NVIp4LOF56jtiKasjet4vGOAwingWqd55cCiILFgjxeAaqnnfLLKQDmM933KUmaw1T8lfftXeld8szR45Wv-SpJdQTe4uWkg/file [following]
-

### Step 3: Preprocess the dataset
This step will preprocess the original dataset in the format that MOMA can train. For this, type
```Rscript preprocess_dataset.R Dataset/Ecomics.transcriptome.no_avg.v8.txt Dataset/Ecomics.transcriptome_with_meta.avg.v8.txt```

Note that the code reads information from <code>Dataset/Meta.txt</code>, <code>Dataset/Meta.Medium.txt</code>, <code>Dataset/Meta.Strain.txt</code>. This will save the preprocessed dataset in the file <code>Dataset/Ecomics.transcriptome_with_meta.avg.v8.txt</code>


In [17]:
!cd MOMA/Dataset && curl ftp://ftp.broadinstitute.org/outgoing/biocyc/Ecomics.transcriptome_with_meta.avg.v8.txt.bz2 > Ecomics.transcriptome_with_meta.avg.v8.txt.bz2

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13.5M  100 13.5M    0     0  4632k      0  0:00:03  0:00:03 --:--:-- 4589k


In [0]:
!cd MOMA/Dataset && bunzip2 Ecomics.transcriptome_with_meta.avg.v8.txt.bz2


### Step 4: Run MOMA
Then you can run MOMA (prediction of transcriptomic response from characteristics of experimental condition) by

```python3 run_moma.py Dataset/Ecomics.transcriptome_with_meta.avg.v8.txt Dataset/GRN.txt OPTIMIZATION_METHOD CONDITION_INDEX_TO_TEST```

* <code>Dataset/Ecomics.transcriptome_with_meta.avg.v8.txt</code> is the dataset to be used for leave-one-condition-out cross-validation.
* <code>Dataset/GRN.txt</code> is the list of gene-regulatory relations (gene-regulatory network). This information is used to regulate the recurrent weight matrix. That is, we constrain the weight matrix not to have nonzero weights on any connections between genes that are not in the gene-regulatory network.
* <code>OPTIMIZATION_METHOD</code> can be SGD or RMSProp. To speed up the model training, RMSprop is recommended for <code>OPTIMIZATION_METHOD</code>.
* <code>CONDITION_INDEX_TO_TEST</code> is the index of condition (that is, a row index, ranging from 0 to 492 as there are 493 conditions or 493 rows in the <code>Dataset/Ecomics.transcriptome_with_meta.avg.v8.txt</code>) to test its prediction from the model that is built on the rest of conditions (Leave-one-condition-out cross-validation; refer to [Kim et al. Nature commms 2016](https://www.nature.com/articles/ncomms13090) for more information).

Please note the following before running the model:
* Note that some of test conditions will not produce prediction results if the conditions are cross-validatable (for example, strain of the test condition is JM109 but this strain is not in the training data).
* The prediction results will display in the console in PCC metric (that is, PCC between predicted expression levels and known expression levels for the test condition) in comparison to the wildtype baseline (that is, PCC between mean expression levels of wildtype profiles and known expression levels for the test condition).


In [25]:
!cd MOMA && python3 run_moma.py Dataset/Ecomics.transcriptome_with_meta.avg.v8.txt Dataset/GRN.txt SGD 455



Using TensorFlow backend.
[DATA READ] 493 samples, 4 timesteps, 521 features
[MODEL CREATED]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 4096)              18915328  
Total params: 18,915,328
Trainable params: 18,915,328
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/1000
492/492 [==============================] - 13s 27ms/step - loss: 0.1160 - mean_squared_error: 0.1160
Epoch 2/1000
492/492 [==============================] - 11s 23ms/step - loss: 0.1132 - mean_squared_error: 0.1132
Epoch 3/1000
492/492 [==============================] - 11s 23ms/step - loss: 0.1132 - mean_squared_error: 0.1132
Epoch 4/1000
492/492 [==============================] - 11s 23ms/step - loss: 0.1132 - mean_squared_error: 0.1132
Epoch 5/1000
492/492 [==============================] - 11s 22ms/step - loss: 0.1132 - mean_squared_erro

In [23]:
!ls -l MOMA/Dataset

total 301712
-rw-r--r-- 1 root root 270852299 Jul 13 18:58 Ecomics.transcriptome.no_avg.v8.txt
-rw-r--r-- 1 root root  37080542 Jul 13 19:33 Ecomics.transcriptome_with_meta.avg.v8.txt
-rw-r--r-- 1 root root    157143 Jul 13 18:54 GRN.txt
-rw-r--r-- 1 root root     39881 Jul 13 18:54 Meta.Medium.txt
-rw-r--r-- 1 root root     36321 Jul 13 18:54 Meta.Strain.txt
-rw-r--r-- 1 root root    770436 Jul 13 18:54 Meta.txt


In [38]:
transcriptome = pd.read_table('MOMA/Dataset/Ecomics.transcriptome.no_avg.v8.txt')
transcriptome['Strain'] = transcriptome['Cond'].str.split('.').str.get(0)
transcriptome['Medium'] = transcriptome['Cond'].str.split('.').str.get(1)
transcriptome['Stress'] = transcriptome['Cond'].str.split('.').str.get(2)
transcriptome['GP'] = transcriptome['Cond'].str.split('.').str.get(3)
transcriptome.set_index(['Strain','GP','Medium','Stress','ID','Cond'])




m.b4412  \
Strain   GP                         Medium Stress         ID    Cond                                                    
MG1655   na_WT                      MD026  RP-overexpress T0568 MG1655.MD026.RP-overexpress.na_WT            0.085563   
                                                          T0569 MG1655.MD026.RP-overexpress.na_WT            0.085563   
                                                          T0570 MG1655.MD026.RP-overexpress.na_WT            0.085563   
DH5alpha na_WT                      MD064  none           T1297 DH5alpha.MD064.none.na_WT                    0.039532   
CG2      na_WT                      MD064  none           T1301 CG2.MD064.none.na_WT                         0.058716   
                                                          T1305 CG2.MD064.none.na_WT                         0.020921   
W3110    hns(delete)93-1_VAR        MD001  Indole         T1315 W3110.MD001.Indole.hns(delete)93-1_VAR       0.067421   
MG1655   r+m+_RM                    MD018  none           T2781 MG1655.MD018.none.r+m+_RM                    0.085563   
                                                          T2782 MG1655.MD018.none.r+m+_RM                    0.085563   
         r-m+_RM                    MD018  none           T2783 MG1655.MD018.none.r-m+_RM                    0.098442   
                                                          T2784 MG1655.MD018.none.r-m+_RM                    0.085563   
         r-m-_RM                    MD018  none           T2785 MG1655.MD018.none.r-m-_RM                    0.085563   
                                                          T2786 MG1655.MD018.none.r-m-_RM                    0.085563   
         na_WT                      MD018  none           T0003 MG1655.MD018.none.na_WT                      0.085563   
                                                          T0004 MG1655.MD018.none.na_WT                      0.085563   
                                                          T0005 MG1655.MD018.none.na_WT                      0.085563   
                                           Cfs            T0015 MG1655.MD018.Cfs.na_WT                       0.085563   
                                                          T0016 MG1655.MD018.Cfs.na_WT                       0.085563   
                                           Mcn            T0024 MG1655.MD018.Mcn.na_WT                       0.085563   
                                                          T0025 MG1655.MD018.Mcn.na_WT                       0.085563   
                                           Cfs;Mcn        T0033 MG1655.MD018.Cfs;Mcn.na_WT                   0.085563   
                                                          T0034 MG1655.MD018.Cfs;Mcn.na_WT                   0.085563   
                                                          T0035 MG1655.MD018.Cfs;Mcn.na_WT                   0.085563   
W3110    na_WT                      MD016  none           T0187 W3110.MD016.none.na_WT                       0.117882   
                                    MD018  none           T0197 W3110.MD018.none.na_WT                       0.085563   
                                                          T0198 W3110.MD018.none.na_WT                       0.085563   
                                                          T0212 W3110.MD018.none.na_WT                       0.085563   
JM109    na_WT                      MD018  none           T0550 JM109.MD018.none.na_WT                       0.079736   
                                                          T0551 JM109.MD018.none.na_WT                       0.072127   
         irrE_OE                    MD018  none           T0560 JM109.MD018.none.irrE_OE                     0.094353   
...                                                                                                               ...   
MG1655   na_WT                      MD047  none           T6009 MG1655.MD047.none.na_WT                      0.060655

In [28]:
!cd MOMA/Dataset && wget -c https://www.dropbox.com/s/nh4d2iuj2tgjhkf/ecomics.proteome.v5.csv

--2018-07-13 20:47:18--  https://www.dropbox.com/s/nh4d2iuj2tgjhkf/ecomics.proteome.v5.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.82.1, 2620:100:6032:1::a27d:5201
Connecting to www.dropbox.com (www.dropbox.com)|162.125.82.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/nh4d2iuj2tgjhkf/ecomics.proteome.v5.csv [following]
--2018-07-13 20:47:19--  https://www.dropbox.com/s/raw/nh4d2iuj2tgjhkf/ecomics.proteome.v5.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc796f937a702ac09400b82a89e8.dl.dropboxusercontent.com/cd/0/inline/ALTz2XNoErvmy-uAzntk66ZfTPlTi-0O9-Zss5Q4Cds1N08RHJF_Ez2GaB6zDcH9Q8BM3QrYhkB2lxyfVro2yd4qPPIqj9aSz71d-JQs_50hVWcRH2FON1GxKTgdHTXgzZBLsvgZu1IFTg35JmUK0NH0PyoeamJLxYTvIHxSP-L7c2KEQ4xzaFp9-hK-UZ1hgn4/file [following]
--2018-07-13 20:47:20--  https://uc796f937a702ac09400b82a89e8.dl.dropboxusercontent.com/cd/0/inline/ALTz2XNoErvmy-u

In [31]:
!ls -l MOMA/Dataset


total 301832
-rw-r--r-- 1 root root    115705 Jul 13 20:47 ecomics.proteome.v5.csv
-rw-r--r-- 1 root root 270852299 Jul 13 18:58 Ecomics.transcriptome.no_avg.v8.txt
-rw-r--r-- 1 root root  37080542 Jul 13 19:33 Ecomics.transcriptome_with_meta.avg.v8.txt
-rw-r--r-- 1 root root    157143 Jul 13 18:54 GRN.txt
-rw-r--r-- 1 root root     39881 Jul 13 18:54 Meta.Medium.txt
-rw-r--r-- 1 root root     36321 Jul 13 18:54 Meta.Strain.txt
-rw-r--r-- 1 root root    770436 Jul 13 18:54 Meta.txt


In [30]:
import pandas as pd
proteomics = pd.read_csv('MOMA/Dataset/ecomics.proteome.v5.csv',index_col=['Strain','MediumID','Medium','Stress','GP'])
proteomics

m.b0002  m.b0003  \
Strain  MediumID Medium            Stress  GP                            
N3433   MD004    MOPS+Glu(0.4%)    none    none         1.000    0.000   
MG1655  MD002    M9+Gly(40%)       none    none         0.415    0.551   
        MD003    M9+Lac(40%)       none    none         0.215    0.335   
        MD001    M9+Glu(40%)       none    none         0.422    0.602   
                                   butanol none         0.144    0.268   
BW25113 MD066    synthetic+Glu     none    b0756(KO)    0.206    0.310   
                                           b2388(KO)    0.243    0.353   
                                           b0688(KO)    0.165    0.264   
                                           b4025(KO)    0.080    0.166   
                                           b3916(KO)    0.012    0.089   
                                           b1723(KO)    0.224    0.332   
                                           b4232(KO)    0.222    0.328   
                                           b2097(KO)    0.209    0.314   
                                           b4493(KO)    0.181    0.282   
                                           b0755(KO)    0.208    0.313   
                                           b4395(KO)    0.258    0.370   
                                           b1854(KO)    0.201    0.305   
                                           b1676(KO)    0.183    0.285   
                                           b1702(KO)    0.103    0.192   
                                           b1852(KO)    0.193    0.295   
                                           b0767(KO)    0.184    0.285   
                                           b2029(KO)    0.208    0.313   
                                           b3386(KO)    0.140    0.234   
                                           b2914(KO)    0.141    0.236   
                                           b4090(KO)    0.216    0.322   
                                           b2935(KO)    0.215    0.321   
                                           b2465(KO)    0.222    0.329   
                                           b3853(KO)    0.188    0.290   
                                           b0008(KO)    0.226    0.334   
                                           none         0.217    0.324   
W3110   MD018    LB                none    none         0.512    1.000   
NCM3722 MD122    modified MOPS+Glu none    none         0.000    0.288   
        MD123    modified MOPS+Lac none    none         0.046    0.369   

                                                      m.b0004  m.b0008  \
Strain  MediumID Medium            Stress  GP                            
N3433   MD004    MOPS+Glu(0.4%)    none    none         1.000    0.451   
MG1655  MD002    M9+Gly(40%)       none    none         0.283    0.240   
        MD003    M9+Lac(40%)       none    none         0.281    0.235   
        MD001    M9+Glu(40%)       none    none         0.217    0.395   
                                   butanol none         0.239    0.255   
BW25113 MD066    synthetic+Glu     none    b0756(KO)    0.218    0.060   
                                           b2388(KO)    0.248    0.120   
                                           b0688(KO)    0.185    0.052   
                                           b4025(KO)    0.117    0.067   
                                           b3916(KO)    0.062    0.034   
                                           b1723(KO)    0.233    0.117   
                                           b4232(KO)    0.231    0.113   
                                           b2097(KO)    0.220    0.179   
                                           b4493(KO)    0.198    0.220   
                                           b0755(KO)    0.220    0.140   
                                           b4395(KO)    0.260    0.131   
                                           b1854(KO)    0.214    0.080   
                                           b1676(KO)    0